Tom (tbc1) - Part 2 Code & Report, intro to the method section, compilation of group's code and report sections including formatting, conclusions to part 4, various other tweaks and fixes. 

Craig (cd253) - Part 1 Code & part of Report, part of part 4 Code and report.

Sam (ss431) - Part 1 Code & Report. Part of part 4 Code & Report.

Veronika (vv1) - Part 2 Code & part of the introduction, made some suggestions how to improve Part 1.

<h4>Introduction to the problem</h4>

Calculus provides powerful tools for understanding the behaviour of functions, but in many practical applications, we only have access to a finite set of discrete data points. This presents a challenge for constructing continuous functions that accurately represent the underlying relationship between quantities.  

The problem of constructing a continuous function that passes through a set of discrete data points is known as data fitting, and interpolation is a specific type of data fitting that seeks to find a function that passes through a set of data points exactly, rather than just approximately.  

The study of interpolation techniques is fundamental in the field of numerical analysis because it provides a means to construct a continuous function that approximates a given set of discrete data points. In many practical applications, it is often necessary to estimate the value of a function at points where the function is not explicitly defined, based on a finite set of data points.  

There are multiple fields where interpolation is widely applied such as medical imaging (interpolation can be used to estimate values of medical imaging data, such as MRI or CT scans, at locations between the measured data points which is useful for image registration and analysis), finance (pricing of derivatives), and machine learning (interpolation is useful to fill in missing data points in a dataset, which can be later implemented in machine learning tasks such as regression and classification). 
 
Interpolation is a common task, but it's important to proceed with caution to avoid unexpected results. Choosing a suitable interpolation method depends on the level of risk and whether the function exhibits smooth or abrupt behaviour or trends at the extremes of the interval of interpolation. 

Lagrange polynomial interpolation and cubic splines are two common techniques for interpolating data and constructing continuous functions that pass through a set of discrete data points. By using these methods, we can approximate the underlying function and estimate values at intermediate points, even if the original data was obtained only from a finite set of measurements. 

<h4>Introduction to the method</h4>

We will be solving the problems given to us by creating a function that performs piecewise-polynomial interpolation, investigating its convergence and accuracy for various degrees of polynomial and then using it on data provided to us in the brief. 

Piecewise-polynomial interpolation is one of four methods of interpolation we have looked at in the course, and has various pros and cons to its use in comparison to these other methods. Our data in part 4 oscillates quite sharply and rapidly, so piecewise linear interpolation would most likely be a poor choice for this as it has trouble being accurate at ‘bends’ where the gradient sign changes, although it is quite cheap computationally. 

In comparison, piecewise polynomial interpolation can capture these bends by using polynomials with sharp bends themselves. If our interpolation was not piecewise, it would likely struggle a great deal with accuracy for this data, as to have sufficiently many ‘turning points’ in the function, the polynomial would need a very high degree. This would be quite computationally expensive and would also mean that if we wanted to perform accurate cubic interpolation, we would need quite well-behaved data. 

Spline interpolation is a kind of piecewise polynomial interpolation that places a restriction on the continuity of the derivatives, which ensures greater smoothness in the interpolation. In part 4 we compare our approach to that of cubic spline interpolation, but in general spline interpolation will be more accurate the smoother the ‘correct’ function for the data is, and less accurate for more jagged, less smooth data, due to this restriction. 

<h4>Question 1</h4>

This section makes heavy use of numpy for implementation. A function was defined with the requisite parameters as inputs with error handling implemented to ensure these were of the right types. Using the parameters and array broadcasting, the interval for each interpolant was created and using the numpy vandermode function and linear algebra solver the coefficients for each interpolant was found and stored. 

The interpolated values was then found by first finding what interpolant each evaluation point sat closest to the middle point of (or extrapolate if outside the interval). Once this is found the corresponding value could be found using the derived coefficients for the interpolant and the evaluation point. This is repeated for each evaluation point. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

def interpol(deg, xk, yk, xj):
    '''
    Returns piecewise polynomial interpolated data for a set of evaluation points
    
        Parameters:
            deg (int): degree of each polynomial interpolant
            xk (ndarray): x values for the data points range with 'float' type 
            yk (ndarray): y values for the data points range with 'float' type
            xj (ndarray): x values for interpolated y values to be found with 'float' type
            
        Raises:
            TypeError: If the degree given is not an integer or equal to or below 0
            TypeError: If the xk, yk or xj numpy arrays are not 1 dimension
            TypeError: If the input data points for xk and yk are not the same length.
            TypeError: If the xk, yk or xj numpy arrays don't contain integer or float values
            
        Returns:
            yj (ndarray): array of interpolated y values with 'float' type
    '''
    
    #Error handling
    if deg <= 0 or not isinstance(deg,int):
        raise TypeError("The degree of the polynomial must be at least one and an integer value for the first parameter")
    #Uses np.ndim to check the dimension of the numpy array being inputted
    if np.ndim(xk) != 1 or np.ndim(yk) != 1 or np.ndim(xj) != 1:
        raise TypeError("Input data points must be in a 1D numpy array for any of the input arrays")
    if len(xk) != len(yk):
        raise TypeError("Input data points for x and y must be the same length")
    #Testing if arrays contain data types which aren't floats or ints to ensure the input arrays only contain the correct data types
    #Uses dtype parameter of the numpy array to check this
    if np.concatenate((xk, yk, xj)).dtype not in ["int", "float"]:
        raise TypeError("One of the input arrays has an incorrect data type, which is not an integer or a float")
   
    
    #Finds the number of data points (knots)
    N = len(xk)
    
    #Number of interpolants to create, so Nint intervals
    Nint = N - deg
    
    #Creates a 2D array where each row stores an interval used to create the interpolants
    pts = np.arange(deg+1) + np.arange(Nint).reshape(Nint,1)
    
    #Empty 2D array to store coefficients
    a = np.zeros((deg+1,Nint))
    
    #Loop over each interval 
    for i in range(Nint):
        #Compute the vandermonde matrix for the current interval
        van = np.vander(xk[pts[i,:]])
        #Solve the system to find the coefficients for the current interval
        a[:,i] = np.linalg.solve(van, yk[pts[i,:]])
        
    #Getting the powers for the polynomials
    powers = deg - np.arange(deg+1)
    
    #Empty array to store interpolated y values
    yj = np.empty_like(xj)
    
    #Loop over evaluation points
    for i in range(len(xj)):
        #If evaluation points are outside of the interval
        #Set k to extrapolate
        if np.all(xj[i]<xk):
            k = 0
        elif np.all(xj[i]>xk):
            k = N-1
        else:
            #Find k where each evaluation point falls into the interval or is equal to the interval
            k = np.where(((xj[i]<xk[1:]) & (xj[i]>=xk[:-1])) | 
                     ((xk[1:]==xj[i]) & (xj[i]>xk[:-1])))[0][0]
        
        #k is the left hand data point of current subinterval
        #So finding index for each interval to be used based on k
        j = k - (deg//2)
        
        #Ensuring j values falls within ranges of indices for the intervals
        j = max(0,j)
        j = min(j,Nint-1)
        
        #Calculating the interpolated y values, using the coefficients for each interval
        yj[i] = np.sum(a[:,j]*xj[i]**powers)

    return yj

#Validating the function
def f(x):
    return np.exp(x) * np.cos(10*x)

N = 20
r0 = np.linspace(0,1/6,N)

#coordinate transformation to test uneven spacing
x1 = 1/(1-r0) 
y1 = f(x1)

#creating the evaluation points
xj = np.linspace(1,1.2,50)

y2 = interpol(3, x1, y1, xj)
    
#Plotting the exact function versus the interpolated function for uneven data
fig = plt.figure(figsize = (15,10))
ax1 = fig.add_subplot()
ax1.plot(x1, f(x1), label = "Exact function")
ax1.plot(x1, y1, 'kx', mew = 2, label = "unevenly spaced data")
ax1.plot(xj, y2, '.', label = "piecewise polynomial interpolation")
ax1.set_xlabel('x values', fontsize = 16)
ax1.set_ylabel('Interpolated points', fontsize = 16)
ax1.set_title('Exact function vs Piecewise interpolated function', fontsize = 16)
ax1.legend(loc = 'best', bbox_to_anchor = (1,1), prop = {'size' : 16})

#Testing incorrect inputs
#Uncomment each test and lines before it to show error handling
#test1 = interpol(-1, x1, y1, xj)
#x1 = np.reshape(x1, (10,2))
#y1 = np.reshape(y1, (10,2))
#test2 = interpol(3, x1, y1, xj)
#x1 = np.append(x1, 1)
#test3 = interpol(3, x1, y1, xj)
#x1 = np.append(x1, 'string')
#y1 = np.append(y1, 2)
#test4 = interpol(3, x1, y1, xj)

As seen from the graph comparison, our piecewise polynomial function closely approximates the exact function for the interpolated values on unevenly spaced and evenly spaced data points. We evaluated our function using a cubic degree polynomial since it overcomes Runge's phenomenon while maintaining a high level of accuracy. However, this method of interpolation doesn't ensure smoothness at the knots meaning the derivatives at the knots may not be continuous. This could cause some error to exist between our function and the exact function. 

As seen from the testing statements at the bottom, our interpol function is robust in handling incorrect or different user inputs. We confirmed our function can successfully handle negative polynomial degrees, incorrect data types and incorrectly shaped and sized data arrays. 

<h4>Question 2</h4>

For part 2 we created a plot by looping over increasing step number N, or equivalently decreasing step-sizes, ‘h’, for different degree polynomials, and then interpolating our original data for evenly-h-spaced points, noting for each step-size the maximum error occurring in the interpolation. 

We took care to use a numpy set-difference function to ensure that the original data x1 and our h-spaced evaluation points never coincided throughout, as instructed in the brief. Then we plotted the maximum errors with their corresponding h-values for each degree polynomial, and displayed these alongside a plot of those h-values raised to the degree of the polynomial interpolation performed plus one, to demonstrate the convergence. We used a logarithmic y-scale and found that these pairs of lines had the same gradient, only varying in height of the lines, showing that the convergence of the maximum errors was ~ $h^{n+1}$ where n is the degree of the polynomial interpolation performed. 

To further check this point we created a second subplot which divided these maximum errors by $h^{n+1}$ and found that these were approximately but not completely constant, which is as we were expecting to show the aforementioned convergence, as it shows that the maximum errors and h^{n+1} only vary by a constant factor, which is equivalent to saying that the maximum errors is of order $h^{n+1}$.

However there was some degree of variance, which could just be accounted for by the accuracy of the method, and thus that python was making floating point errors due to the tiny size of our maximum errors. This problem was actually worse for some other choices of evaluation points and ranges of N, so we had to tune these as instructed in the brief to find a more robust convergence. In particular, some larger ranges for N resulted in such small ‘h’s that there were worse floating point errors and so quite noisy data, and the same was true for higher degree polynomials, because the more precise our interpolation is, the more the max errors become increasingly tiny.

In [ ]:
import numpy as np

def f(x):
    return np.exp(x)*np.cos(10*x)

N = 20
r0 = np.linspace(0,1/6,N)

x1 = 1/(1-r0) # coordinate transformation to test uneven spacing
y1 = f(x1)


import matplotlib.pyplot as plt
%matplotlib inline
fig = plt.figure(figsize = (15,5))
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)

for deg in range(1,5):
    # Initialise lists to store step-sizes and maximum errors in.
    # These will reset for each degree so that we can plot separately.
    hs = []
    max_error_arr = []
    for N in range(28, 138, 6):
        # Calculate step size
        h = (x1[-1] - x1[0])/(N - 1)
        # Set up evenly spaced data, and calculate function values at these
        # points.
        x_data = np.arange(x1[0], x1[-1], h)
        y_data = f(x_data)
        
        # Now set up evaluation points
        x_eval = np.linspace(x1[0], x1[-1], 100)
        # Use numpy setdiff1d function to acquire a set difference,
        # ensuring data points and evaluation points do not coincide.
        x_eval = np.setdiff1d(x_eval, x_data)
        y_eval = interpol(deg, x_data, y_data, x_eval)

        # Calculate the maximum error
        max_error = max(abs(f(x_eval) - y_eval))

        # Append the step size and the maximum error to hs and max_errors
        hs.append(h)
        max_error_arr.append(max_error)

    # Plot the current degree with a label
    ax1.plot(hs, max_error_arr, label=f'Polynomial of degree {deg}', color=f'C{deg%10}')
    # Plot h^{deg+1} to see convergence for each
    ax1.plot(hs,(np.array(hs)**(deg+1)), color=f'C{deg%10}',
             linestyle ='--', label =f'h^{deg+1}')
    ax1.set_yscale('log')
    ax1.set_xlabel('h')
    ax1.set_ylabel('Max error')
    ax1.set_title('Max error of polynomial interpolation, and h**{deg+1}')
    ax1.legend()
    
    # Plot the max error divided by h**{deg+1} to check if constant to further illustrate convergence
    ax2.plot(hs,np.array(max_error_arr)/(np.array(hs)**(deg+1)),
             color=f'C{deg%10}',label =f'Polynomial degree {deg}')
    ax2.set_yscale('log')
    ax2.set_xlabel('h')
    ax2.set_ylabel('Max error/h**{deg+1}')
    ax2.set_title('(Max error of {deg} degree polynomial interpolation) / h^{deg+1}')
    ax2.legend(loc='best', bbox_to_anchor=(0.5, 0.4, 0.5, 0.5))
    



<h4>Question 3</h4>

This section makes use of Jupyter Widgets to achieve the desired interactivity. The interact function was used after importing the necessary libraries. This function can easily create a slider that will pass the selected value to a function included in the arguments for interact(). 

Example: 
interact(f, x=10) Where f is a previously defined function.

To further customise the slider the min and max values were selected to match the assumed function and a step size of 1 was set to allow only integer values to be passed to the function. 

We have also taken additional steps to ensure that the graphic is easy to use for instance we fixed the y axis for easier comparison between different polynomial degrees. We also used array broadcasting to ensure our code runs efficiently given the high workload required when toggling the slider. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
from ipywidgets import interact, IntSlider

#Define our function
def f(x):
    return np.exp(x) * np.cos(10 * x)

#Some preliminary boundary & scale assumptions
M = 11
x0 = np.linspace(1,2,M)
y0 = f(x0)
N = 101
x = np.linspace(0.9,2.1,N)

#Define the Lagrangian
def lagrange(n,f):
    
    #Choosing which segments to interpolate from
    xlist = []
    length = len(y0)
    xlist = np.append(xlist, x0[range((length//2) - math.floor((n+1)/2), (length//2) + math.ceil((n+1)/2))])
    ylist = f(xlist)
    
    #Solving the Vandermode coefficients
    B = np.vander(xlist)
    b = np.linalg.solve(B,ylist)
    
    #Using array broadcasting to construct the Lagrangian
    pows_ = ((n+1)-1-np.arange((n+1))).reshape((n+1),1)
    xnew_ = np.reshape(x,(1,N))
    ynew_ = np.sum((xnew_**pows_)*b.reshape((n+1),1),axis=0)
    return(ynew_)

#Create an adjustable plotting function
def plot(n):
    fig = plt.figure("Function and interpolant", figsize = (15,5) )
    ax = fig.add_subplot(1, 2, 1)
    ap = fig.add_subplot(1, 2, 2)

    #Polynomial comparison plot
    ax.plot(x0,y0,'-',mew=2,label='data')
    ax.plot(x,lagrange(n,f),'-',label = str('poly of degree ') + str(n))
    ax.set_title('Poly Interpolation of Knots from $f(x) = e^xcos(10x)$')
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_ylim(-10,10) #Helps keep plot stable. Can be changed for different functions.
    ax.legend()

    #Error Plot
    ap.plot(x, abs(f(x)- lagrange(n,f)));
    ap.set_xlabel("x");
    ap.set_ylabel("$abs(f(x) - interpolant)$");
    ap.set_title("Error compared to $f(x)$");

#Create the adjustable slider    
degree_slider = IntSlider(min=1, max=10, step=1, value=0, description='Degree');
interact(plot, n=degree_slider);


interactive(children=(IntSlider(value=1, description='Degree', max=10, min=1), Output()), _dom_classes=('widge…

As we would expect, increasing the degree of the polynomial causes the error of the interpolant to decrease, resulting in a smoother fit to the data. An improvement to the efficiency of the code would be to rework the plotting part so that the interact function only redraws the necessary lines , rather than redrawing the whole graph on every change.  

The decision was made to allow continuous updates to the interact function as this would allow for a faster way to see the changes made to the interpolant as n changes. 

The addition of the error plot gives a better indication of the accuracy for each of the chosen degrees. However, it currently looks unintuitive as the original function in the other plot has not been drawn as a single smooth curve, but rather as a piecewise linear interpolant. 

<h4>Question 4</h4>

The data was loaded using numpy loadtxt() and the missing number of data points could be found. The data was recorded in 0.1s intervals for 30s and so there is expected to be 300 data points. There was found to be 292 so 8 were missing. 

The function from part 1 was called on an array containing the values for the first 20s with a specified degree of 3 and these interpolated values were stored. This was repeated for a natural cubic spline implemented using scipy. Using matplotlib.pyplot these were plotted beside each other and then the differences were plotted. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline

%matplotlib inline

data = np.loadtxt("wave_data1.txt")

missingNumber = 300-len(data[0]) #8
#Getting new sampled data

new_N = 3
#Time values to be evaluated
x1 = np.arange(0.01,20, 0.01)
#remember to use condition to exclude missing data points
new_x0 = data[0]
new_y0 = data[1]

#Interpolated piecewise cubic polynomial values
y1 = interpol(new_N,new_x0,new_y0,x1)

#Interpolated scipy values
cs = CubicSpline(new_x0, new_y0, bc_type = 'natural')
y2 = cs(x1)

#Plotting pt1 interpolation vs scipy interpolation vs f(x)

fig = plt.figure(figsize = (10,5))
ax1 = fig.add_subplot(121)
ax1.plot(x1, y1, color = "red", label = 'Part 1 Piecewise Cubic Interpolation')
ax1.plot(x1, y2, color = "blue", label = 'Scipy Cubic Spline Interpolation')
ax1.set_xlabel('time (s)')
ax1.set_ylabel('elevation (m)')
ax1.set_title('Cubic piecewise vs Scipy Cubic Spline')
ax1.legend(loc = "best", bbox_to_anchor = (1,1))

#Plotting difference between pt1 interpolation and scipy interpolation
fig2 = plt.figure(figsize = (10,5))
ax2 = fig2.add_subplot(122)
ax2.plot(x1, np.abs(y1-y2), label = 'Difference in interpolation methods')
ax2.set_xlabel('time (s)')
ax2.set_ylabel('elevation (m)')
ax2.set_title('Difference between piecewise cubic polynomial and scipy cubic spline interpolation')
ax2.legend(loc = "best", bbox_to_anchor = (1,1))

plt.show()

We found that our own method and the scipy cubic interpolation method produced extremely similar results on the data, with the difference being of the order $10^{-4}$ for much of our data. We can safely assume that the scipy method is at least fairly accurate given its place as a standard, widely used method for interpolation, and so we can infer that our own method is about as accurate as this, although it is hard to know whether it is more so or less so. 

The time-values where we see the largest difference between results of the two methods are those close to or coinciding with times where data is missing from the original dataset. This is to be expected, as it is here that our interpolation methods must do more ‘guessing’. It is hard to tell however which method does a ‘better’ job at interpolating using this fact, as we would need to be able to reference the missing values or some explicit function in order to do so.
